In [1]:
# 필요 라이브러리 호출
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기
df = pd.read_csv('data/naver_comment_2.csv', encoding='utf-8-sig')
df.head()

,title,score,score_num,best_num,best,best_recomm,best_unrecomm,comment_num,comment,star
0,웰캄투실버라이프,9.94,19878,15,아 할머니 귀여우시다 힐링툰 감이 오네요 잘볼게용!\n시리얼에 뜨거운 우유...나만...,52588,599,2220,정주행 시작\n솔찍히 자는데 깨우면 짜증남..할머니라도..맘은 이해하지만 더 자고 ...,9.98
1,원수를 사랑하라,9.95,31528,15,근데 여주 대단하다...엄마랑 아빠가 저런 사람들인데 멀끔히 차려입고 면접보는 거면...,129562,3792,987,근데 나 자신을 믿는것.. 지금 내가 제일 필요한 말임..ㅠㅠㅠㅠㅠ 어제 면접 말려...,9.92
2,약초마을 연쇄살초사건,9.73,10367,15,아니ㅋㅋㅋㅋㅋㅋ작가님ㅋㅋㅋㅋㅋㅋ이제 작물 쪽으로 길을 트신건가요ㅋㅋㅋㅋㅋㅋㅋㅋ\n팀...,63294,255,1462,이거 왤케 재밌냨ㅋ 팀장님 죽었는데 뒷담 까는거 보소\n약초마을 연쇄살인도 아닌 연...,9.87
3,AI가 세상을 지배한다면,9.88,12207,15,그대가 POGO 싶었소\n라움 코드 자기가 넣었다고... 이리 당당하게 선전POGO...,59238,967,2036,완결보고 정주행만 3번째 볼때마다 감정이 다름 영화로 나오기엔 잘라내거나 스토리에 ...,9.94
4,닥터 프로스트 시즌 3~4,9.95,30579,15,아무생각없이 내렸다가 이거 보고 놀란사람 손!!!\n헐?\n제가 이 웹툰을 보면서 ...,119758,2817,9707,퍄ㅑㅑ\n닥프 1기 2기 무료로 풀려있으니까 꼭보세요\n아 이런 띵작은 유료되기전에...,9.97


In [3]:
okt = Okt()
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text, stem=True)
    return tokens_ko

import re

def hangul(text):
    return re.sub("[^가-힣ㄱ-하-ㅣ\\s]", "", text)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def train_to_evaluate(X, y, file_name, max_words=10000, maxlen=30, embedding_dim=100, *args):
    
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    X_seq = tokenizer.texts_to_sequences(X)

    X_pad = pad_sequences(X_seq, maxlen=maxlen)
    y = np.asarray(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
    
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
    model.add(Flatten())
    for unit in args:
        model.add(Dense(unit, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    cp = ModelCheckpoint(filepath='data/'+file_name,
                         monitor='accuracy',
                         save_best_only=True)
    
    es = EarlyStopping(monitor='accuracy',
                       patience=15)
    
    
    hist = model.fit(X_train, y_train,
                     epochs=100, batch_size=32,
                     validation_data=(X_val, y_val),
                     callbacks=[cp, es],
                     verbose=0)
    
    
    model.load_weights('data/'+file_name)
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
#     print('MAE :', np.round(mae, 4), 'RMSE :', np.round(np.sqrt(mse), 4))
        
    return model, hist, loss, accuracy

In [5]:
# 9.9 이상, 미만 비율이 6:4
def category(star):
    if star >= 9.9:
        return 1
    else:
        return 0

# 베댓만

## 1. Dense 층 없이

In [7]:
X1 = df['best']
y = df['star'].apply(category)

X1 = X1.apply(okt_tokenizer).values

In [8]:
from tqdm import tqdm

bestmodel = None
bestacc = 0
bestmxwords = None
bestmxlen = None
bestembdim = None
bestunit = None
result_df = pd.DataFrame(columns=['maxwords', 'maxlen', 'embedding_dim', 'loss', 'accuracy'])

for mxwords in tqdm(range(10, 100+1, 10)):
    for mxlen in [20, 25, 30]:
        for embdim in range(10, 100+1, 5):
            model = train_to_evaluate(X1, y, 'embedding1_category.h5', mxwords, mxlen, embdim)
            result_df.loc[len(result_df)] = [mxwords, mxlen, embdim, model[2], model[3]]
            if bestacc < model[3]:
                bestmodel = model
                bestacc = model[3]
                bestmxwords = mxwords
                bestmxlen = mxlen
                bestembdim = embdim
                
bestmodel[0].save('data/embedding1_category.h5')

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [38:53<00:00, 233.35s/it]


In [9]:
print(f'Accuracy: {bestmodel[3]:.4f}, loss: {bestmodel[2]:.4f}')
print(f'--bestmxwords: {bestmxwords}, --bestmxlen: {bestmxlen}, --bestembdim: {bestembdim}')

Accuracy: 0.6518, loss: 0.7934
--bestmxwords: 40, --bestmxlen: 25, --bestembdim: 10


In [10]:
result_df.sort_values(by=['loss']).head(10)

,maxwords,maxlen,embedding_dim,loss,accuracy
16,10.0,20.0,90.0,0.756928,0.526786
0,10.0,20.0,10.0,0.777922,0.580357
561,100.0,30.0,60.0,0.784888,0.526786
386,70.0,30.0,40.0,0.788699,0.580357
190,40.0,25.0,10.0,0.793409,0.651786
31,10.0,25.0,70.0,0.796037,0.562500
1,10.0,20.0,15.0,0.796811,0.562500
480,90.0,25.0,35.0,0.804651,0.535714
565,100.0,30.0,80.0,0.807502,0.544643
492,90.0,25.0,95.0,0.809602,0.526786


In [11]:
result_df.sort_values(by=['accuracy'], ascending=False).head(10)

,maxwords,maxlen,embedding_dim,loss,accuracy
190,40.0,25.0,10.0,0.793409,0.651786
173,40.0,20.0,20.0,0.899858,0.642857
186,40.0,20.0,85.0,0.883529,0.642857
193,40.0,25.0,25.0,0.887582,0.633929
187,40.0,20.0,90.0,0.870686,0.633929
135,30.0,25.0,20.0,1.134634,0.625000
181,40.0,20.0,60.0,0.856500,0.625000
180,40.0,20.0,55.0,0.848120,0.625000
174,40.0,20.0,25.0,0.910253,0.616071
185,40.0,20.0,80.0,0.849257,0.616071


## 특문 제거

In [12]:
X1a = df['best'].apply(hangul)
X1a = X1a.apply(okt_tokenizer).values

In [13]:
bestmodel = None
bestacc = 0
bestmxwords = None
bestmxlen = None
bestembdim = None
bestunit = None
result_df = pd.DataFrame(columns=['maxwords', 'maxlen', 'embedding_dim', 'loss', 'accuracy'])

for mxwords in tqdm(range(10, 100+1, 10)):
    for mxlen in [20, 25, 30]:
        for embdim in range(10, 100+1, 5):
            model = train_to_evaluate(X1a, y, 'embedding1a_category.h5', mxwords, mxlen, embdim)
            result_df.loc[len(result_df)] = [mxwords, mxlen, embdim, model[2], model[3]]
            if bestacc < model[3]:
                bestmodel = model
                bestacc = model[3]
                bestmxwords = mxwords
                bestmxlen = mxlen
                bestembdim = embdim
                
bestmodel[0].save('data/embedding1a_category.h5')

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [39:37<00:00, 237.76s/it]


In [14]:
print(f'Accuracy: {bestmodel[3]:.4f}, loss: {bestmodel[2]:.4f}')
print(f'--bestmxwords: {bestmxwords}, --bestmxlen: {bestmxlen}, --bestembdim: {bestembdim}')

Accuracy: 0.6786, loss: 0.7849
--bestmxwords: 60, --bestmxlen: 30, --bestembdim: 80


In [15]:
result_df.sort_values(by=['loss']).head(10)

,maxwords,maxlen,embedding_dim,loss,accuracy
554,100.0,30.0,25.0,0.715474,0.616071
6,10.0,20.0,40.0,0.722290,0.526786
12,10.0,20.0,70.0,0.737564,0.535714
568,100.0,30.0,95.0,0.745339,0.589286
5,10.0,20.0,35.0,0.745827,0.517857
0,10.0,20.0,10.0,0.750044,0.526786
569,100.0,30.0,100.0,0.751053,0.598214
1,10.0,20.0,15.0,0.753073,0.526786
561,100.0,30.0,60.0,0.753302,0.607143
566,100.0,30.0,85.0,0.757537,0.553571


In [16]:
result_df.sort_values(by=['accuracy'], ascending=False).head(10)

,maxwords,maxlen,embedding_dim,loss,accuracy
337,60.0,30.0,80.0,0.784903,0.678571
438,80.0,30.0,15.0,0.972068,0.651786
331,60.0,30.0,50.0,0.798838,0.651786
553,100.0,30.0,20.0,0.784109,0.642857
494,90.0,30.0,10.0,0.887830,0.642857
503,90.0,30.0,55.0,0.800649,0.642857
313,60.0,25.0,55.0,0.904156,0.642857
319,60.0,25.0,85.0,0.925158,0.642857
495,90.0,30.0,15.0,0.778993,0.633929
312,60.0,25.0,50.0,0.865542,0.633929


## 2. Dense 층 추가

In [17]:
bestmodel = None
bestacc = 0
bestmxwords = None
bestmxlen = None
bestembdim = None
bestunit = None
result_df = pd.DataFrame(columns=['maxwords', 'maxlen', 'embedding_dim', 'unit', 'loss', 'accuracy'])

for mxwords in tqdm(range(20, 60+1, 10)):
    for mxlen in [10, 15, 20, 25, 30]:
        for embdim in range(10, 100+1, 5):
            for unit in [16, 32, 48, 64]:
                model = train_to_evaluate(X1, y, 'embedding1b_category.h5', mxwords, mxlen, embdim, unit)
                result_df.loc[len(result_df)] = [mxwords, mxlen, embdim, unit, model[2], model[3]]
                if bestacc < model[3]:
                    bestmodel = model
                    bestacc = model[3]
                    bestmxwords = mxwords
                    bestmxlen = mxlen
                    bestembdim = embdim
                    bestunit = unit
                
bestmodel[0].save('data/embedding1b_category.h5')

100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [1:35:16<00:00, 1143.30s/it]


In [18]:
print(f'Accuracy: {bestmodel[3]:.4f}, loss: {bestmodel[2]:.4f}')
print(f'--bestmxwords: {bestmxwords}, --bestmxlen: {bestmxlen}, --bestembdim: {bestembdim}, --bestunit: {bestunit}')

Accuracy: 0.6696, loss: 0.9026
--bestmxwords: 40, --bestmxlen: 20, --bestembdim: 50, --bestunit: 64


In [19]:
result_df.sort_values(by=['loss']).head(10)

,maxwords,maxlen,embedding_dim,unit,loss,accuracy
1024,40.0,25.0,55.0,16.0,0.815479,0.616071
1013,40.0,25.0,40.0,32.0,0.833425,0.598214
1061,40.0,25.0,100.0,32.0,0.854336,0.589286
1007,40.0,25.0,30.0,64.0,0.860727,0.660714
950,40.0,20.0,55.0,48.0,0.869630,0.633929
1040,40.0,25.0,75.0,16.0,0.870210,0.589286
1075,40.0,30.0,20.0,64.0,0.873789,0.607143
1018,40.0,25.0,45.0,48.0,0.894071,0.642857
1011,40.0,25.0,35.0,64.0,0.895694,0.598214
1051,40.0,25.0,85.0,64.0,0.897124,0.589286


In [20]:
result_df.sort_values(by=['accuracy'], ascending=False).head(10)

,maxwords,maxlen,embedding_dim,unit,loss,accuracy
947,40.0,20.0,50.0,64.0,0.902617,0.669643
946,40.0,20.0,50.0,48.0,0.928112,0.660714
1007,40.0,25.0,30.0,64.0,0.860727,0.660714
769,40.0,10.0,20.0,32.0,1.626126,0.651786
812,40.0,10.0,75.0,16.0,1.471952,0.651786
1829,60.0,30.0,15.0,32.0,1.002640,0.651786
1001,40.0,25.0,25.0,32.0,0.943214,0.651786
785,40.0,10.0,40.0,32.0,1.638763,0.651786
936,40.0,20.0,40.0,16.0,1.101996,0.651786
1305,50.0,20.0,25.0,32.0,1.157982,0.642857


## 특문 제거

In [21]:
bestmodel = None
bestacc = 0
bestmxwords = None
bestmxlen = None
bestembdim = None
bestunit = None
result_df = pd.DataFrame(columns=['maxwords', 'maxlen', 'embedding_dim', 'unit', 'loss', 'accuracy'])

for mxwords in tqdm(range(20, 60+1, 10)):
    for mxlen in [10, 15, 20, 25, 30]:
        for embdim in range(10, 100+1, 5):
            for unit in [16, 32, 48, 64]:
                model = train_to_evaluate(X1a, y, 'embedding1c_category.h5', mxwords, mxlen, embdim, unit)
                result_df.loc[len(result_df)] = [mxwords, mxlen, embdim, unit, model[2], model[3]]
                if bestacc < model[3]:
                    bestmodel = model
                    bestacc = model[3]
                    bestmxwords = mxwords
                    bestmxlen = mxlen
                    bestembdim = embdim
                    bestunit = unit
                
bestmodel[0].save('data/embedding1c_category.h5')

100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [1:37:59<00:00, 1175.89s/it]


In [22]:
print(f'Accuracy: {bestmodel[3]:.4f}, loss: {bestmodel[2]:.4f}')
print(f'--bestmxwords: {bestmxwords}, --bestmxlen: {bestmxlen}, --bestembdim: {bestembdim}, --bestunit: {bestunit}')

Accuracy: 0.7054, loss: 0.9835
--bestmxwords: 60, --bestmxlen: 25, --bestembdim: 25, --bestunit: 48


In [23]:
result_df.sort_values(by=['loss']).head(10)

,maxwords,maxlen,embedding_dim,unit,loss,accuracy
1846,60.0,30.0,35.0,48.0,0.927900,0.633929
1848,60.0,30.0,40.0,16.0,0.947456,0.598214
1854,60.0,30.0,45.0,48.0,0.947719,0.633929
1862,60.0,30.0,55.0,48.0,0.953246,0.598214
1844,60.0,30.0,35.0,16.0,0.962221,0.616071
1887,60.0,30.0,85.0,64.0,0.963433,0.616071
1768,60.0,25.0,35.0,16.0,0.964076,0.598214
1137,40.0,30.0,100.0,32.0,0.965037,0.491071
1488,50.0,30.0,65.0,16.0,0.967874,0.553571
1850,60.0,30.0,40.0,48.0,0.974204,0.607143


In [24]:
result_df.sort_values(by=['accuracy'], ascending=False).head(10)

,maxwords,maxlen,embedding_dim,unit,loss,accuracy
1762,60.0,25.0,25.0,48.0,0.983517,0.705357
1830,60.0,30.0,15.0,48.0,1.106802,0.669643
1812,60.0,25.0,90.0,16.0,1.014190,0.651786
1836,60.0,30.0,25.0,16.0,1.016309,0.651786
396,30.0,10.0,30.0,16.0,1.568066,0.651786
421,30.0,10.0,60.0,32.0,1.573885,0.642857
427,30.0,10.0,65.0,64.0,1.374246,0.642857
1752,60.0,25.0,15.0,16.0,1.196743,0.642857
445,30.0,10.0,90.0,32.0,1.521430,0.642857
1801,60.0,25.0,75.0,32.0,1.053362,0.642857


# 테스트

In [29]:
'''
Accuracy: 0.6786, loss: 0.7849
--bestmxwords: 60, --bestmxlen: 30, --bestembdim: 80
'''
max_words = 60
maxlen = 30

from tensorflow.keras.models import load_model

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X1a)
X_seq = tokenizer.texts_to_sequences(X1a)
X_pad = pad_sequences(X_seq, maxlen=maxlen)
y = np.asarray(y)

X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [47]:
model = load_model('data/embedding1a_category.h5')

In [49]:
model.evaluate(X_test, y_test)

5/5 [==============================] - 0s 2ms/step - loss: 1.0031 - accuracy: 0.5071


[1.003082275390625, 0.5071428418159485]

In [51]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(df['best'], df['star'], test_size=0.2, random_state=0)

In [52]:
pred = model.predict(X_test)

In [53]:
y_test_orig[y_test_orig < 9.8]

432    9.50
266    9.57
251    9.72
103    8.81
674    9.69
215    8.90
312    9.44
608    9.70
532    8.72
406    8.38
162    9.54
496    9.75
279    9.53
478    8.80
200    9.78
568    7.02
267    8.96
477    9.56
242    9.76
417    9.75
466    9.22
229    9.47
294    9.41
245    9.76
634    8.63
261    9.66
37     6.28
666    9.79
181    9.73
272    9.75
465    7.22
210    9.65
230    9.74
310    9.52
644    9.78
Name: star, dtype: float64

In [54]:
y_test_orig_copy = y_test_orig.reset_index(drop=True)

In [55]:
y_test_orig_copy[y_test_orig_copy < 9.8]

3      9.50
6      9.57
8      9.72
9      8.81
10     9.69
12     8.90
13     9.44
21     9.70
27     8.72
32     8.38
34     9.54
36     9.75
37     9.53
42     8.80
44     9.78
47     7.02
53     8.96
56     9.56
61     9.76
67     9.75
73     9.22
78     9.47
79     9.41
81     9.76
84     8.63
93     9.66
94     6.28
106    9.79
115    9.73
119    9.75
129    7.22
130    9.65
133    9.74
135    9.52
136    9.78
Name: star, dtype: float64

In [56]:
X_test_orig.iloc[129]

'언제나 오덕으로 나오는 캐릭터는 안경에,주근깨에,사회생활 부적응자에 다키마쿠라껴안고다니면서 하악거리는건 똑같네.애니,만화 좋아하는게 아이돌 좋아하는거랑뭐 크게 다른줄아시나요...웹툰보면 아이돌좋아하는 애는 예쁘고 친구들도 많은데애니좋아하는 사람만 이렇게 나오는거 이제지친다...\n나는 별론데.. .덕후가 저렇게 표현되는건 좀.. .\n저도 오덕으로써 한마디 하겠습니다.일단 덕후들 다 쿠션가지고 하앜 거리지 않아요그리고 일본어를 조금씩은 할 줄 알겠지만 아침부터 히사시부리 라면서 이목끌지는 않아요...애니를 본다고 차별하는게 너무 싫어서 이 웹툰을 보고 눈이 찌푸려 졌어요 ㅜㅜㅜ그래도 아직 1화고 더 성장하길 것 같아요.아 그리고 이기자 그린 잘봤습니다!!\n요리고 이어서 만화고?\n여기서 메갈이 왜나와 ,,? ㅋㅋㅋㅋ 너무 예민하게 반응하네 ; 심하다\n믿기지 않으시겠지만 이분이 기자 이야기 쓰신분입니다..\n나도 덕후디만 저런식으로 표현되는거 싫어 이런것들 때문에 그냥 애니 보는거만 좋아하는 사람들한태도 이상한 인식 박혀서 맨날 욕먹고 팩트폭행해줘도 맨날 덕후는 믿고 거른다면서 무심가고 나보다 못한놈들이\n덕후 이미지를 안여돼+~했다능+캐릭터 베개 들고 하악 거리는 사람으로 박아놓는 것 같아서 별로다.. 선입견만 모아둔 완전체 극혐.\n공감 고민툰 급은 아니지만 덕후에 대한 편견이 심하여 조치가 필요함.\n덕후에 대한 스테레오타입들을 일부로 싸그리 모아서 외적으로 나타낸 캐릭터니까 당연히 저렇게 그려지죠... 아직 1화고 주인공의 배경도 상세히 나오지 않은 마당에 벌써부터 단정하는건 섣부른 판단이 아닐까요? 작가로써는 이 사람이 우리가 생각하는 오타쿠다 라고 표현하고싶었는데 난데없이 엄청 예쁘고 섹시한 여주가 나오면 독자들이 내가 원하는 주인공의 모습을 오해하는건 아닐까, 하고 생각해서 이렇게 그렸을 수도 있어요.그리고 댓글에서 서로의 의견 표현은 자유로울 수 있으나 사적 감정이 개입되면 비방과 싸움이 일어날 수 있으니 다들 조심하시는게 좋을 것 같습니다.

In [57]:
y_test_orig.iloc[129], pred[129]

(7.22, array([0.9599828], dtype=float32))

In [58]:
X_test_orig.iloc[47]

'솔직히 내용 다 까먹어서 정주행하려햇는데 유료화돼서 못본사람 손\n휴재의하늘\n아니 무슨 11개월만에 왔는데 죄송하단 말한번없어.\n맨날 휴재하더니 이제 휴재하면 안볼래요 그냥.\n시즌3을 기억못하시는분들을 위해 써요심연의 하늘은 시즌3때 하늘로 생각했던 사다리를 올라오고 표지판을 보게되죠 도시는 소용돌이같이 갑자기 쓸어가고 어느날은 구멍도 한번에 파이게 되죠 그러고 둘은 떨어지게 되고 여자아이는 헬리콥터도타다가 하늘을 계속 찾게 됩니다. 그러다가 결국 둘은 만나고 휴재되었죠.작가님 휴재기간이 길면 내용파악이 안되서 힘들어요\nㅋㅋㅋ 누가 돈이 없어서 유료화가 짜증나는거냐?? ㅋㅋㅋ 뭔 200원가지고 있는척들이야 ㅋㅋㅋ 돈이 없어서 짜증나는게 아니라 작자 하는 꼬라지가 짜증나는거다\n여기 사람들이 지금 결제하는 돈이 아깝다고 글쓰는거 아니잖아난독증 있는 사람들 몇명 있는거 같네처음 스마트툰 연재때부터 보던 사람들은 알겠지만원래 이 웹툰 원래 휴재 오지게 땡기던 웹툰임근데 이번에는 그 기간도 길었을 뿐더러작가 매일 술마시고 노는 영상이 페북에 올라다녔다는데자기 작품 완결도 안낸상태에서 장기간 휴재해놓고 스토어에 올려버리면 아무리 애정갖고 보던 독자들도 좀 어이없지않겠냐?\n차원만화 보고있었는데 나오니까 하나더 생겻다\n마지막에 중국인이 둘이 만난거보고 미사일 떨궜음\n왜 유로화함 기억하지도 멋하는데;\n술먹을돈 떨어졌냐? 어차피 돈좀벌리면 또 휴재할꺼 그냥평생하지마라 그 따위로 일할거면\n앞의 내용이 전혀 기억나지 않는다둘다 인간이긴 하냐?\n도대체 연재만 몇번을 봐야하는지 이렇게 30화정도 내고 또 끊길듯\n시즌3마지막 기억이안나ㅠㅠ 설명충등판좀..\n처음에 작가의 말 . 이었답니다. 댓글 보고 고친거래요. 아무려면 그냥 베댓이 됐겠어요?? 작가가 댓글 보고 고치는 바람에 베댓되신분만 갈수록 싫어요가 많아지는 게 보여서 한마디 남깁니다.원래는 . 이었다는 댓글이 베댓만 올라갔어도 베댓된 분이 이처럼 욕을 먹진 않을텐데 말이죠.뒤늦게 본 사람들이 단지 작가의 

In [59]:
y_test_orig.iloc[47], pred[47]

(7.02, array([0.91515267], dtype=float32))

In [60]:
X_test_orig.iloc[94]

'밑그림 남아있는건 좀 너무하지 않냐 한 컷도 아니고 대부분의 컷이 그럼\n네웹 어디 학원에서 넣어준다는 소문 있던데 진짜인가 보네...\n털선을 자기 아이덴티티로 밀고 갈 생각인진 모르겠지만 이건 그림을 잘 그리고 못 그리기 이전에 완성도와 성의의 문제임. 연습장에 그리다 만 그림같은 퀄리티를 네이버에서 정식연재한다고?\n선 정리 조금만 해주면 진짜 좋을것 같은데 그점 빼고는 다 좋은거 같네요\n내용이랑 그림체는 많이안나쁜데..잡선이 너무많이보이네요 파일잘못 올리셨낭..?\n작가님 한텐 좀 죄송하긴 하다만 솔직히 네이버 요즘 일 안하는거같음 베도만 가 봐도 그림이쁘고 깔끔하고 스토리 탄탄한거 좀만 둘러봐도 나오는데。。。 이건 좀 아닌것같음\n작가가털선쓰는건첨보네\n이런 그림은 개성이 아니라 못그린거야 개성은 다른거지\n진짜 죄송한데..이건 뭐 느낌이 있는 그림도 아닌거 같구 털선 너무 보기가 싫음 털선임..가끔 털선이여도 예쁜 털선이 있는데 이건 걍 털선 그 자체.. 털선때문에 내용에 집중을 못하겠음..\n뭐야.. 베도에 이거보다 퀄 좋고 스토리 구성 좋은 작품 많은데.. 왜 베도도 아니고 이런 퀄을 그리는 작가들을 데리고 오는거지?\n털선 이어붙이기가 좀 심함 그림 그리는 사람이라면 털선은 모두가 싫어하는데\n그림을 학원에서 배우던 독학을 하던 먼저 해야하는건 선 연습이에요.그 기초도 안되어있으니 사람들의 시선이 불편하죠..\n밑밑밑댓 미술 안 배워본듯. 선 좀 지저분한 걸 떠나서 구도 인체비율 투시 틀린 거 많음 ㅋㅋ 당장 배경만 해도 투시 다 틀렸는데? 앵글에 맞지 않게 사람 넣은 컷도 보이고. 인체비율은 작가 개성이라고 봐도 문제될 거 없지만 배경 투시는 진짜 수학책 입체도형 자료 급으로 위아래 선이 평행한데 뭔 투시가 맞아 ㅋㅋ 소실점이 어딨는지도 못찾겠더라 난.. 그리고 물타기물타기 하는데 오히려 지금 작화 감싸는 게 더 물타기같다고 생각됨. 개성있고 보기 좋다고? 아니 못그리는 걸 못그린다고 말하는게 어떻게 물타기야 ㅋㅋㅋ 보기 싫으면 보지 말라

In [61]:
y_test_orig.iloc[94], pred[94]

(6.28, array([0.15195438], dtype=float32))

이 또한 결과는 별로인 것 같다.  
1화 베댓으로 평점을 예측한다는 가정이 틀린 걸까